In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


점수를 높이기 위해 어떤 예상을 했는지

본인 팀이 어떤 규칙을 정했는지

어떤 모델을 사용했는지

발표 3분

In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

### 전처리

Name : Mr. , Mrs. , Miss, Master

Cabin : Null. , NotNull(Crew).



Ticket, Embarked : 버려.

In [3]:
# cabin data를 indexing
# null 이면 0, 아니면 1 (crew라는 의미)

def Cabin_1(col):
    cabin = col
    if pd.isnull(cabin):
        return 0
    else:
        return 1
    
train['Cabin'] = train['Cabin'].apply(Cabin_1)

In [4]:
# name data를 indexing

def Name_1(col):
    name = col
    if 'Mr.' in name:
        return 0
    elif 'Mrs' in name:
        return 1
    elif 'Miss' in name:
        return 2
    elif 'Master' in name:
        return 3
    else:
        return 4
    
train['Name'] = train['Name'].apply(Name_1)

In [5]:
# sex data를 indexing
# male = 1, female = 0

train.Sex = train.groupby('Sex').ngroup()

In [6]:
# Age data의 null값 impute.

def Age_1(cols) :
    Age=cols[0]
    Pclass=cols[1] 
    
    if pd.isnull(Age) :
        if Pclass==1 :
            return 42 
        elif Pclass==2:
            return 25
        else :
            return 22
    else :
        return Age
    
train.Age=train[['Age','Pclass']].apply(Age_1,axis=1)

In [7]:
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,0,1,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,1,0,38.0,1,0,PC 17599,71.2833,1,C
2,3,1,3,2,0,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,1,0,35.0,1,0,113803,53.1000,1,S
4,5,0,3,0,1,35.0,0,0,373450,8.0500,0,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,4,1,27.0,0,0,211536,13.0000,0,S
887,888,1,1,2,0,19.0,0,0,112053,30.0000,1,S
888,889,0,3,2,0,22.0,1,2,W./C. 6607,23.4500,0,S
889,890,1,1,0,1,26.0,0,0,111369,30.0000,1,C


In [8]:
features = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin']

X_train = train[features]
y_train = train.Survived

In [9]:
# Scaling.

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)
x_train_scaled = scaler.transform(X_train)

print(x_train_scaled)

[[1.         0.         1.         ... 0.         0.01415106 0.        ]
 [0.         0.25       0.         ... 0.         0.13913574 1.        ]
 [1.         0.5        0.         ... 0.         0.01546857 0.        ]
 ...
 [1.         0.5        0.         ... 0.33333333 0.04577135 0.        ]
 [0.         0.         1.         ... 0.         0.0585561  1.        ]
 [1.         0.         1.         ... 0.         0.01512699 0.        ]]


In [10]:
#Logistic Regression으로 훈련.

from sklearn.linear_model import LogisticRegression

lgr=LogisticRegression(max_iter=500)
lgr_train=lgr.fit(x_train_scaled,y_train)

In [11]:
# test data 전처리 중 null 값 체크.

test['Cabin'] = test['Cabin'].apply(Cabin_1)

test['Name'] = test['Name'].apply(Name_1)

test.Sex = test.groupby('Sex').ngroup()

test.Age=test[['Age','Pclass']].apply(Age_1,axis=1)

test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           1
Cabin          0
Embarked       0
dtype: int64

In [12]:
test[test.Fare.isnull() == True]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,0,1,60.5,0,0,3701,NaN,0,S


In [13]:
# 해당 클래스의 평균 fare로 impute.

test = test.fillna(test[test.Pclass==3].Fare.mean())

In [14]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [15]:
# test data로 scaling해서 예측값 구하기.

X_test = test[features]

test_scaler = MinMaxScaler()
test_scaler.fit(X_test)
x_test_scaled = test_scaler.transform(X_test)

y_pred = lgr.predict(x_test_scaled)

In [16]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [17]:
pId = pd.DataFrame(test.PassengerId)
surv = pd.DataFrame(y_pred, columns=['Survived'])
y_submit = pd.concat([pId,surv], axis=1)

y_submit

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [18]:
y_submit.to_csv("Submission.csv", index = False)

# 제출본 확인.

## Bagging 방법 활용, ~DecisionTree

In [19]:
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


params = [{'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}]
b_clf=BaggingClassifier(GridSearchCV(DecisionTreeClassifier(random_state=42),params,cv=3,verbose=1),n_estimators=100,max_samples=100,bootstrap=True,n_jobs=-1)
b_clf.fit(x_train_scaled,y_train)


y_pred=b_clf.predict(x_test_scaled)

# estimator 100회, 500회, 1000회 로 돌려봤으나 의미 없었음.

In [20]:
pId = pd.DataFrame(test.PassengerId)
surv = pd.DataFrame(y_pred, columns=['Survived'])
y_submit = pd.concat([pId,surv], axis=1)

y_submit

y_submit.to_csv("New_Submission.csv", index = False)

# 제출본 확인.